# Spring-Mass System: Harmonic Oscillation and Damping

## Theoretical Background

The spring-mass system is a fundamental model in classical mechanics that demonstrates simple harmonic motion (SHM) and serves as the basis for understanding oscillatory phenomena across physics and engineering.

### Equation of Motion

Consider a mass $m$ attached to a spring with spring constant $k$. When displaced from equilibrium by a distance $x$, the restoring force follows **Hooke's Law**:

$$F = -kx$$

Applying Newton's second law $F = ma = m\ddot{x}$, we obtain the equation of motion:

$$m\ddot{x} + kx = 0$$

### Undamped Oscillation

The general solution for the undamped case is:

$$x(t) = A\cos(\omega_0 t + \phi)$$

where the **natural angular frequency** is:

$$\omega_0 = \sqrt{\frac{k}{m}}$$

The **period** of oscillation is:

$$T = \frac{2\pi}{\omega_0} = 2\pi\sqrt{\frac{m}{k}}$$

### Damped Oscillation

In realistic systems, damping forces (proportional to velocity) dissipate energy. The equation becomes:

$$m\ddot{x} + c\dot{x} + kx = 0$$

where $c$ is the damping coefficient. Defining the **damping ratio**:

$$\zeta = \frac{c}{2\sqrt{km}}$$

The system exhibits three regimes:
- **Underdamped** ($\zeta < 1$): Oscillatory decay
- **Critically damped** ($\zeta = 1$): Fastest non-oscillatory return to equilibrium
- **Overdamped** ($\zeta > 1$): Slow exponential decay

For the underdamped case, the solution is:

$$x(t) = Ae^{-\zeta\omega_0 t}\cos(\omega_d t + \phi)$$

where the **damped frequency** is:

$$\omega_d = \omega_0\sqrt{1 - \zeta^2}$$

### Energy Considerations

The total mechanical energy in an undamped system oscillates between kinetic and potential forms:

$$E = \frac{1}{2}m\dot{x}^2 + \frac{1}{2}kx^2 = \frac{1}{2}kA^2$$

In damped systems, this energy decays exponentially:

$$E(t) = E_0 e^{-2\zeta\omega_0 t}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# System parameters
m = 1.0      # mass (kg)
k = 4.0      # spring constant (N/m)
omega_0 = np.sqrt(k/m)  # natural frequency

# Initial conditions
x0 = 1.0     # initial displacement (m)
v0 = 0.0     # initial velocity (m/s)

# Time array
t = np.linspace(0, 10, 1000)

print(f"Natural frequency: ω₀ = {omega_0:.3f} rad/s")
print(f"Period: T = {2*np.pi/omega_0:.3f} s")

In [ ]:
def spring_mass_ode(y, t, m, c, k):
    """
    ODE system for damped spring-mass system.
    
    Parameters:
    -----------
    y : array [x, v]
        State vector (position, velocity)
    t : float
        Time
    m : float
        Mass
    c : float
        Damping coefficient
    k : float
        Spring constant
    
    Returns:
    --------
    dydt : array [dx/dt, dv/dt]
    """
    x, v = y
    dxdt = v
    dvdt = -(c/m)*v - (k/m)*x
    return [dxdt, dvdt]

# Define damping scenarios
damping_cases = {
    'Undamped (ζ=0)': 0.0,
    'Underdamped (ζ=0.1)': 0.1,
    'Underdamped (ζ=0.3)': 0.3,
    'Critically damped (ζ=1)': 1.0,
    'Overdamped (ζ=2)': 2.0
}

# Solve for each case
solutions = {}
y0 = [x0, v0]

for label, zeta in damping_cases.items():
    c = 2 * zeta * np.sqrt(k * m)  # damping coefficient
    sol = odeint(spring_mass_ode, y0, t, args=(m, c, k))
    solutions[label] = sol
    print(f"{label}: c = {c:.3f} N·s/m")

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Color scheme
colors = plt.cm.viridis(np.linspace(0, 0.9, len(damping_cases)))

# Plot 1: Position vs Time
ax1 = axes[0, 0]
for (label, sol), color in zip(solutions.items(), colors):
    ax1.plot(t, sol[:, 0], label=label, color=color, linewidth=1.5)
ax1.set_xlabel('Time (s)', fontsize=11)
ax1.set_ylabel('Position x (m)', fontsize=11)
ax1.set_title('Displacement vs Time', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right', fontsize=8)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='k', linewidth=0.5)

# Plot 2: Velocity vs Time
ax2 = axes[0, 1]
for (label, sol), color in zip(solutions.items(), colors):
    ax2.plot(t, sol[:, 1], label=label, color=color, linewidth=1.5)
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Velocity v (m/s)', fontsize=11)
ax2.set_title('Velocity vs Time', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linewidth=0.5)

# Plot 3: Phase Space (x vs v)
ax3 = axes[1, 0]
for (label, sol), color in zip(solutions.items(), colors):
    ax3.plot(sol[:, 0], sol[:, 1], label=label, color=color, linewidth=1.5)
    # Mark initial point
    ax3.plot(sol[0, 0], sol[0, 1], 'o', color=color, markersize=6)
ax3.set_xlabel('Position x (m)', fontsize=11)
ax3.set_ylabel('Velocity v (m/s)', fontsize=11)
ax3.set_title('Phase Space Portrait', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right', fontsize=8)
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='k', linewidth=0.5)
ax3.axvline(x=0, color='k', linewidth=0.5)

# Plot 4: Energy vs Time
ax4 = axes[1, 1]
for (label, sol), color in zip(solutions.items(), colors):
    x_vals = sol[:, 0]
    v_vals = sol[:, 1]
    # Total mechanical energy: E = (1/2)mv² + (1/2)kx²
    KE = 0.5 * m * v_vals**2
    PE = 0.5 * k * x_vals**2
    E_total = KE + PE
    ax4.plot(t, E_total, label=label, color=color, linewidth=1.5)

ax4.set_xlabel('Time (s)', fontsize=11)
ax4.set_ylabel('Energy (J)', fontsize=11)
ax4.set_title('Total Mechanical Energy vs Time', fontsize=12, fontweight='bold')
ax4.legend(loc='upper right', fontsize=8)
ax4.grid(True, alpha=0.3)
ax4.set_ylim(bottom=0)

plt.suptitle('Spring-Mass System: Effects of Damping', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

# Save the figure
plt.savefig('plot.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print("\nPlot saved to 'plot.png'")

## Analysis and Observations

### Key Results

1. **Undamped System ($\zeta = 0$)**: The mass oscillates indefinitely with constant amplitude. In phase space, this traces a perfect ellipse, indicating conservation of mechanical energy.

2. **Underdamped Systems ($\zeta < 1$)**: Oscillations persist but with exponentially decaying amplitude. The envelope follows $e^{-\zeta\omega_0 t}$. Phase trajectories spiral inward toward the origin.

3. **Critically Damped ($\zeta = 1$)**: The system returns to equilibrium in the minimum time without oscillating. This is optimal for applications requiring fast settling (e.g., door closers, shock absorbers).

4. **Overdamped ($\zeta > 1$)**: The return to equilibrium is slower than critical damping, with no oscillations. The phase trajectory approaches the origin along a curve without crossing either axis.

### Energy Dissipation

The energy plot clearly shows:
- Undamped: Constant energy (horizontal line)
- Damped: Exponential decay following $E(t) = E_0 e^{-2\zeta\omega_0 t}$

### Practical Applications

- **Vehicle suspensions**: Tuned to be slightly underdamped for comfort
- **Seismometers**: Critically damped for accurate response
- **Building oscillations**: Understanding resonance for earthquake engineering